In [ ]:
! nvidia-smi

In [ ]:
! pip install kaggle tensorboard wget
! pip install tensorboard --upgrade
! tensorboard --logdir=runs

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d gverzea/edible-wild-plants
! unzip /content/edible-wild-plants.zip

In [ ]:
!git clone https://github.com/alexfrst/data-augmentation-benchmark

In [ ]:
import sys

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

sys.path.append("data-augmentation-benchmark")

import model.model as model
from torch.optim import lr_scheduler
from model.evaluate import evaluate
from model.train import train_model
from utils.dataset import load_dataset
from utils.print_utils import Symbols
from utils.tensorboard import tb_writer


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Working on {device} {Symbols.OK}")

train_loader, val_loader, test_loader, nb_classes = load_dataset("datasets/dataset", "datasets/dataset-test", batch_size=128)


Working on cuda:0 ✓
Loading dataset...
Nombre d'images de train : 5902
Nombre d'images de val : 656
Nombre d'images de test : 310
Apprentissage sur 62 classes
Data loading and integrity check done ✓
Apprentissage en transfer learning


Epoch 0: 11it [02:35, 13.21s/it, Acc (val)=0.161, loss train=5.42, loss val=3.81]

In [ ]:
inception_v3 = model.load_inception(nb_classes)
criterion = nn.CrossEntropyLoss()

params_to_update = model.get_params_tranfer_learning(inception_v3, transfer_learning=True)

optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=5, factor=0.5, min_lr=1e-5)


print("Apprentissage en transfer learning avec inception")
inception_v3.to(device)
inception_v3.train(True)

best_model, best_val_score =  train_model(inception_v3, train_loader, val_loader, scheduler, criterion, evaluate, 65, device=device, tb_writer=tb_writer, training_name="inception_v3_tl")
inception_v3.train(False)
loss, accuracy = evaluate(best_model, test_loader, device, criterion)

print("Accuracy (test): %.1f%%" % (100 * accuracy))

In [ ]:
torch.cuda.empty_cache()
inception_v3 = model.load_inception(nb_classes, transfer_learning=False)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(inception_v3.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=5, factor=0.5, min_lr=1e-5)


print("Apprentissage en fine tunning avec inception")
inception_v3.to(device)
inception_v3.train(True)

best_model, best_val_score =  train_model(inception_v3, train_loader, val_loader, scheduler, criterion, evaluate, 65, device=device, tb_writer=tb_writer, training_name="inception_v3_ft")
inception_v3.train(False)
loss, accuracy = evaluate(best_model, test_loader, device, criterion)

print("Accuracy (test): %.1f%%" % (100 * accuracy))

In [ ]:
torch.cuda.empty_cache()
mobilenet = model.load_mobilenet(nb_classes, transfer_learning=True)
criterion = nn.CrossEntropyLoss()

params_to_update = model.get_params_tranfer_learning(mobilenet)

optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=5, factor=0.5, min_lr=1e-5)


print("Apprentissage en transfer learning avec mobilenet")
mobilenet.to(device)
mobilenet.train(True)

best_model, best_val_score =  train_model(mobilenet, train_loader, val_loader, scheduler, criterion, evaluate, 65, device=device, tb_writer=tb_writer, training_name="mobilenet_tl")
mobilenet.train(False)
loss, accuracy = evaluate(best_model, test_loader, device, criterion)

print("Accuracy (test): %.1f%%" % (100 * accuracy))

In [ ]:
mobilenet = model.load_mobilenet(nb_classes, transfer_learning=False)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(mobilenet.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=5, factor=0.5, min_lr=1e-5)


print("Apprentissage en fine tunning avec mobilenet")
mobilenet.to(device)
mobilenet.train(True)

best_model, best_val_score =  train_model(mobilenet, train_loader, val_loader, scheduler, criterion, evaluate, 65, device=device, tb_writer=tb_writer, training_name="mobilenet_ft")
mobilenet.train(False)
loss, accuracy = evaluate(best_model, test_loader, device, criterion)

print("Accuracy (test): %.1f%%" % (100 * accuracy))

In [1]:
!tensorboard dev upload --logdir runs --name "My latest experiment"  --description "Simple comparison of several hyperparameters"

^C
